In [3]:
import tensorflow as tf
import numpy as np

# Basic Operations

## Placeholders, constants, operations and session

A `placeholder` returns a `Tensor` that may be used as a handle for feeding a value, but not evaluated directly.

`shape` and `name` are optional.

In [17]:
a = tf.placeholder(tf.float32, shape=(), name= 'a')
b = tf.placeholder(tf.float32, shape=(), name='b')

A `placeholder_with_default` returns a `Tensor` that has the same type as `input`. It is a placeholder tensor that defaults to `input` if it is not fed.

In [23]:
c = tf.placeholder_with_default(5.0, shape=(), name='c')

Another type of node is a `constant`. Like all TensorFlow constants, it takes no inputs, and it outputs a value it stores internally. This `Tensor` can be evaluated directly, it doesn't need to be fed a value.

In [29]:
d = tf.constant(4.5, tf.float32)

Define basic symbolic operations.

An `Operation` is a node in a TensorFlow `Graph` that takes zero or more `Tensor` objects as input, and produces zero or more `Tensor` objects as output.

For example `mul = tf.multiply(a, b)` creates an `Operation` of type `multiply` that takes tensors `a` and `b` as input, and produces `mul` as output.

In [34]:
mul = tf.multiply(a, b)
mul_with_default = tf.multiply(a, c)
mul_with_constant =tf.multiply(a, d)
mul_short = a * b # Shortcut

A `Session` is a class for running TensorFlow operations.

A `Session` object encapsulates the environment in which `Operation` objects are executed, and `Tensor` objects are evaluated.

In [36]:
with tf.Session() as sess:
    print("Multiply two tensors: {}".format(sess.run(mul, feed_dict={a: 2, b: 3})))
    print("Multiply with a default placeholder: {}".format(sess.run(mul_with_default, feed_dict={a:2})))
    print("Multiply with a default placeholder taht is fed a value: {}".format(sess.run(mul_with_default, feed_dict={a:2, c:8})))
    print("Multiply with a constant: {}".format(sess.run(mul_with_constant, feed_dict={a:2})))
    print("Multiply two tensors using the short definition for the node: {}".format(sess.run(mul_short, feed_dict={a:2, b:3})))

Multiply two tensors: 6.0
Multiply with a default placeholder: 10.0
Multiply with a default placeholder taht is fed a value: 16.0
Multiply with a constant: 9.0
Multiply two tensors using the short definition for the node: 6.0
